In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle
import numpy as np

In [8]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
#removing unecessary columns
data = data.drop(columns=['RowNumber','CustomerId','Surname'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
#Observing the unique values of columns with text values- so that they can be converted to numerical values
print(data['Geography'].unique())
print(data['Gender'].unique()) 
print(data['Tenure'].unique()) #based on this in app.py max value is set as 10 and min value is set as 0
print(data['NumOfProducts'].unique()) #based on this in app.py max value is set as 4 and min value is set as 1

['France' 'Spain' 'Germany']
['Female' 'Male']
[ 2  1  8  7  4  6  3 10  5  9  0]
[1 3 2 4]


In [11]:
#encoding
gender_encoder = LabelEncoder()
data['Gender'] = gender_encoder.fit_transform(data[['Gender']])
geo_encoder = OneHotEncoder(sparse_output=False)
geo_data = geo_encoder.fit_transform(data[['Geography']]) #once geo_encoder is used to perform transformation- geo_encoder stores the feature name information
geo_data_df = pd.DataFrame(geo_data,columns=geo_encoder.get_feature_names_out())#since in encoder the sparse_output is set as False - geo_data is not a matrix but an array output - hence, we need not convert it into an array- when using it in the dataframe function

c:\ANN_implementation\venv_new\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
#Removing original Geography column in dataset and adding the encoded geography data
data = pd.concat([data.drop(columns=['Geography']),geo_data_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
#Pickle file 
#Pickling converts an object into a byte stream - serializes an object structure and saves it on disk. 
#Thy byte stream has all the information to reconstruct the object- deserialized to its original form and used in other python scripts

with open('gender_encoder_file.pkl','wb') as file:
    pickle.dump(gender_encoder,file)

with open('geo_encoder_file.pkl','wb') as file:
    pickle.dump(geo_encoder,file)


In [14]:
#Dependent and independent  features - Output will be dependent feature- inputs will be independent features
X = data.drop(columns=['Exited'])
y = data['Exited']

In [15]:
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0


In [16]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [17]:
#train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42) 
#random state range can be between 0-42
X_train


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
9254,686,1,32,6,0.00,2,1,1,179093.26,1.0,0.0,0.0
1561,632,1,42,4,119624.60,2,1,1,195978.86,0.0,1.0,0.0
1670,559,1,24,3,114739.92,1,1,0,85891.02,0.0,0.0,1.0
6087,561,0,27,9,135637.00,1,1,0,153080.40,1.0,0.0,0.0
6669,517,1,56,9,142147.32,1,0,0,39488.04,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,768,1,54,8,69712.74,1,1,1,69381.05,1.0,0.0,0.0
5191,682,0,58,1,0.00,1,1,1,706.50,1.0,0.0,0.0
5390,735,0,38,1,0.00,3,0,0,92220.12,1.0,0.0,0.0
860,667,1,43,8,190227.46,1,1,0,97508.04,1.0,0.0,0.0


In [18]:
#Scaling inputs
input_scaler = StandardScaler()
X_train = input_scaler.fit_transform(X_train)
X_test = input_scaler.fit_transform(X_test)
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [19]:
#saving the standard scaler to pickle file
with open('input_scaler_file.pkl','wb') as file:
    pickle.dump(input_scaler,file)

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import datetime


In [21]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1 is interconnected with the input layer
    # The number of neurons in the input layer is same as the number of features in X,i.e.,number of columns in X
    Dense(32,activation='relu'),
   Dense(1,activation='sigmoid')
])
model.summary()

c:\ANN_implementation\venv_new\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.compile(optimizer=Adam(learning_rate=0.01),loss =BinaryCrossentropy(),metrics=['accuracy'])

In [23]:
#Tensorboard
#Tensorboard is for visulaizing all logs during training- ensures that logs are created and stored
#The same can be done manually with matplotlib
log_dir = 'logs/fit'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #logs to a file name with the current date-time mentioned 
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1) #histogram frequency is none by default. histogram_freq=1 ensures that histogram is computed for every epoch

In [24]:
#EarlyStopping - used to stop training after certain epochs if there are no improvements in the monitored parameter(specified by parameter "monitor")
#the improvements can be increasing or decreasing in nature and its dependent on "mode" parameter
#mode="auto" by default- which means its measured as increasing or decreasing based on the monitored parameter- mode can also be "min" or "max"
#monitor = "val_loss" by default
#patience parameter is the number of epochs after which the training has to be stopped if no improvements are observed. 
# By default- patience=0 - i.e., it does not give time to the parameter for improvement and stops right after no improvement is observed in the monitored parameter
#restore_best_weights - defines whether model weights will be restored from the best epoch of the monitored quantity after training. restore_best_weights=False by default.
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True)

In [25]:
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
    )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8086 - loss: 0.4502 - val_accuracy: 0.8530 - val_loss: 0.3582
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8553 - loss: 0.3505 - val_accuracy: 0.8545 - val_loss: 0.3501
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8570 - loss: 0.3525 - val_accuracy: 0.8600 - val_loss: 0.3504
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8637 - loss: 0.3396 - val_accuracy: 0.8590 - val_loss: 0.3402
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8616 - loss: 0.3337 - val_accuracy: 0.8600 - val_loss: 0.3511
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8651 - loss: 0.3331 - val_accuracy: 0.8620 - val_loss: 0.3383
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8612 - loss: 0.3322 - val_accuracy: 0.8565 - val_loss: 0.3454
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8627 - loss: 0.3314 - val_accu

In [26]:
#model save
#if you're using keras-.h5 format is perfect for preserving the model architecture as well as weights. It does not require the model architecture to be reinstantiated when loaded
model.save('model.h5')

In [27]:
#loading tensorboard extension
%load_ext tensorboard

In [31]:
%tensorboard --logdir C:/ANN_implementation/logs/fit20250530-192027

Reusing TensorBoard on port 6007 (pid 20948), started 0:01:57 ago. (Use '!kill 20948' to kill it.)